# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

'''
Import note: The classroom currently uses sklearn version 0.19.
If you need to use an imputer, it is available in sklearn.preprocessing.Imputer,
instead of sklearn.impute as in newer versions of sklearn.
'''

'\nImport note: The classroom currently uses sklearn version 0.19.\nIf you need to use an imputer, it is available in sklearn.preprocessing.Imputer,\ninstead of sklearn.impute as in newer versions of sklearn.\n'

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [2]:
# Load in the general demographics data.
azdias = pd.read_csv("Udacity_AZDIAS_Subset.csv", delimiter = ";")

# Load in the feature summary file.
feat_info = pd.read_csv("AZDIAS_Feature_Summary.csv", delimiter = ";")

In [3]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).
azdias.head(2)

,AGER_TYP,ALTERSKATEGORIE_GROB,ANREDE_KZ,CJT_GESAMTTYP,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_VORSORGER,FINANZ_ANLEGER,FINANZ_UNAUFFAELLIGER,FINANZ_HAUSBAUER,...,PLZ8_ANTG1,PLZ8_ANTG2,PLZ8_ANTG3,PLZ8_ANTG4,PLZ8_BAUMAX,PLZ8_HHZ,PLZ8_GBZ,ARBEIT,ORTSGR_KLS9,RELAT_AB
0,-1,2,1,2.0,3,4,3,5,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,1,2,5.0,1,5,2,5,4,5,...,2.0,3.0,2.0,1.0,1.0,5.0,4.0,3.0,5.0,4.0


In [4]:
azdias.tail(5)

,AGER_TYP,ALTERSKATEGORIE_GROB,ANREDE_KZ,CJT_GESAMTTYP,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_VORSORGER,FINANZ_ANLEGER,FINANZ_UNAUFFAELLIGER,FINANZ_HAUSBAUER,...,PLZ8_ANTG1,PLZ8_ANTG2,PLZ8_ANTG3,PLZ8_ANTG4,PLZ8_BAUMAX,PLZ8_HHZ,PLZ8_GBZ,ARBEIT,ORTSGR_KLS9,RELAT_AB
891216,-1,3,2,5.0,1,4,2,5,4,4,...,3.0,2.0,0.0,0.0,1.0,2.0,3.0,NaN,NaN,NaN
891217,-1,2,1,4.0,3,3,3,2,2,3,...,1.0,3.0,3.0,2.0,4.0,5.0,3.0,4.0,6.0,5.0
891218,-1,2,2,4.0,2,4,2,5,4,3,...,4.0,2.0,0.0,0.0,1.0,3.0,4.0,2.0,2.0,3.0
891219,-1,1,1,3.0,1,5,3,5,5,5,...,1.0,4.0,3.0,1.0,5.0,1.0,1.0,4.0,7.0,5.0
891220,-1,4,1,1.0,4,2,5,2,1,5,...,3.0,3.0,1.0,0.0,1.0,4.0,4.0,3.0,4.0,5.0


In [5]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Data columns (total 85 columns):
AGER_TYP                 891221 non-null int64
ALTERSKATEGORIE_GROB     891221 non-null int64
ANREDE_KZ                891221 non-null int64
CJT_GESAMTTYP            886367 non-null float64
FINANZ_MINIMALIST        891221 non-null int64
FINANZ_SPARER            891221 non-null int64
FINANZ_VORSORGER         891221 non-null int64
FINANZ_ANLEGER           891221 non-null int64
FINANZ_UNAUFFAELLIGER    891221 non-null int64
FINANZ_HAUSBAUER         891221 non-null int64
FINANZTYP                891221 non-null int64
GEBURTSJAHR              891221 non-null int64
GFK_URLAUBERTYP          886367 non-null float64
GREEN_AVANTGARDE         891221 non-null int64
HEALTH_TYP               891221 non-null int64
LP_LEBENSPHASE_FEIN      886367 non-null float64
LP_LEBENSPHASE_GROB      886367 non-null float64
LP_FAMILIE_FEIN          886367 non-null float64
LP_FAMILIE_GROB          886367 n

In [6]:
azdias.describe()

,AGER_TYP,ALTERSKATEGORIE_GROB,ANREDE_KZ,CJT_GESAMTTYP,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_VORSORGER,FINANZ_ANLEGER,FINANZ_UNAUFFAELLIGER,FINANZ_HAUSBAUER,...,PLZ8_ANTG1,PLZ8_ANTG2,PLZ8_ANTG3,PLZ8_ANTG4,PLZ8_BAUMAX,PLZ8_HHZ,PLZ8_GBZ,ARBEIT,ORTSGR_KLS9,RELAT_AB
count,891221.000000,891221.000000,891221.000000,886367.000000,891221.000000,891221.000000,891221.000000,891221.000000,891221.000000,891221.000000,...,774706.000000,774706.000000,774706.000000,774706.000000,774706.000000,774706.000000,774706.000000,794005.000000,794005.000000,794005.00000
mean,-0.358435,2.777398,1.522098,3.632838,3.074528,2.821039,3.401106,3.033328,2.874167,3.075121,...,2.253330,2.801858,1.595426,0.699166,1.943913,3.612821,3.381087,3.167854,5.293002,3.07222
std,1.198724,1.068775,0.499512,1.595021,1.321055,1.464749,1.322134,1.529603,1.486731,1.353248,...,0.972008,0.920309,0.986736,0.727137,1.459654,0.973967,1.111598,1.002376,2.303739,1.36298
min,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.00000
25%,-1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,3.000000,2.000000,2.000000,2.000000,...,1.000000,2.000000,1.000000,0.000000,1.000000,3.000000,3.000000,3.000000,4.000000,2.00000
50%,-1.000000,3.000000,2.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,2.000000,3.000000,2.000000,1.000000,1.000000,4.000000,3.000000,3.000000,5.000000,3.00000
75%,-1.000000,4.000000,2.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,...,3.000000,3.000000,2.000000,1.000000,3.000000,4.000000,4.000000,4.000000,7.000000,4.00000
max,3.000000,9.000000,2.000000,6.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,4.000000,4.000000,3.000000,2.000000,5.000000,5.000000,5.000000,9.000000,9.000000,9.00000


#### The majority of columns has missing values.

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [7]:
# Identify missing or unknown data values and convert them to NaNs.
def list_convert(line):
    line = line.replace("[", "")
    line = line.replace("]", "")
    a = line.split(",")
    return a
feat_info["missing_or_unknown"] = feat_info["missing_or_unknown"].apply(list_convert)

In [8]:
feat_info["missing_or_unknown"].value_counts()

[-1]          26
[-1, 9]       17
[-1, 0]       16
[0]           12
[]            10
[-1, 0, 9]     1
[-1, X]        1
[-1, XX]       1
[XX]           1
Name: missing_or_unknown, dtype: int64

In [9]:
# Sets the attribute as index so it is easier to handle with the .loc mehtod
feat_info.set_index("attribute", inplace = True)

In [10]:
type(feat_info["missing_or_unknown"][0])

list

In [ ]:
# Iterates through the columns of azdias
for column in azdias.columns:
    # saves the values as a list which are nan in this colum
    null_values = feat_info.loc[str(column), "missing_or_unknown"]
    for entry in null_values:
        # Some columns do not have an indicator for null values,
        try:
            if (entry != ""):
                # replaces every null value with an np.nan in the column
                azdias[column] = azdias[column].replace(int(entry), np.nan)
            else:
                print("Entry was an empty string: -{}- (should be empty)".format(entry))
        except:
            print("Entry {} is not a number.".format(entry))
            azdias[column] = azdias[column].replace(entry, np.nan)

In [ ]:
# check if it worked
azdias.head(5)

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
# Create a list with the number of nan values for each column
nan_values = []
for column in azdias.columns:
    number_of_nan = (len(azdias[column]) - azdias[column].count())/len(azdias[column])*100
    nan_values.append(number_of_nan)

In [ ]:
# Create a large figure
plt.figure(figsize = (40,15))
# Draw bar graph with column labels on the x-axis and the corresponding number of nan values as height
plt.bar(azdias.columns,nan_values);
plt.xticks(rotation = 70);

In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)
nan_columns = ["AGER_TYP", "GEBURTSJAHR", "TITEL_KZ", "ALTER_HH", "HH_EINKOMMEN_SCORE"]
# drop columns
azdias.drop(axis = 1, columns = nan_columns, inplace = True)

In [ ]:
# check
len(azdias.columns)

#### Discussion 1.1.2: Assess Missing Data in Each Column

Columns regarding the age of the person seem to have a lot of missing data.
The columns with over 30% missing data are removed from the dataset:
 - AGER_TYP
 - GEBURTSJAHR
 - TITEL_KZ
 - ALTER_HH
 - HH_EINKOMMEN_SCORE

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
azdias.shape[1]

In [ ]:
# How much data is missing in each row of the dataset?
azdias["missing"] = azdias.apply(lambda x: (azdias.shape[1] - x.count()), axis=1)

In [ ]:
len(azdias.query("missing == 0"))*100/len(azdias)

In [ ]:
azdias["missing"].plot.hist();

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.
df_full = azdias.query("missing == 0")
df_miss = azdias.query("missing != 0")

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.
def plot_comparison(column):
    plt.figure(figsize = (15,5))
    plt.subplot(1,2,1)
    sns.countplot(df_full[column], color = "grey")
    plt.xlabel("No missing data")
    plt.title(column)
    plt.subplot(1,2,2)
    sns.countplot(df_miss[column], color = "grey")
    plt.xlabel("Missing data")
    plt.title(column);

In [ ]:
plot_comparison("FINANZ_SPARER")

In [ ]:
plot_comparison(azdias.columns[12])

In [ ]:
plot_comparison(azdias.columns[20])

In [ ]:
plot_comparison(azdias.columns[22])

In [ ]:
plot_comparison(azdias.columns[40])

#### Discussion 1.1.3: Assess Missing Data in Each Row

The rows with missing data have very different distributions of values. This can be observed in the above 5 bar graphs. It is recommended to use only the subset of data that has no missing data.

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?
cat_mixed_info = feat_info.query("type == ['categorical', 'mixed']")

In [ ]:
cat_mixed_info

In [ ]:
len(cat_mixed_info[cat_mixed_info["type"] == "mixed"])

In [ ]:
len(cat_mixed_info[cat_mixed_info["type"] == "categorical"])

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?
cat_columns = list(feat_info.query("type == ['categorical']").index)


In [ ]:
one_hot = []

for column in cat_columns:
    try:
        print("Column {} has {} unique values".format(column,len(df_full[column].unique())))
        # If it is already one hot encoded do nothing
        if list(df_full[column].unique()) == [0, 1]:
            pass
        # Otherwise append it to a list for one-hot encoding
        else:
            one_hot.append(column)
    except KeyError:
        print("The column {} got removed in an earlier step".format(column))

In [ ]:
df_oh = df_full.copy()

In [ ]:
one_hot

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.
df_oh = pd.get_dummies(data = df_oh, columns = list(one_hot))


In [ ]:
df_oh.shape

In [ ]:
df_oh.head()

#### Discussion 1.2.1: Re-Encode Categorical Features

I checked all categorical columns if they were already one-hot encoded. In case they were not I added them to a list of columns to be transformed. In a last step I used the pandas get_dummies() method to one-hot encode all categorical columns.

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
# numbers associated with mainstream movement
mainstream = [1,3,5,8,10,12,14]

df_oh["MAINSTREAM"] = 2

In [ ]:
for number in mainstream:
    df_oh.loc[df_oh.PRAEGENDE_JUGENDJAHRE == number, 'MAINSTREAM'] = "1"
df_oh["MAINSTREAM"].replace(2, 0, inplace = True)

In [ ]:
df_oh[["PRAEGENDE_JUGENDJAHRE", "MAINSTREAM"]].head(5)

In [ ]:
df_oh["GENERATION"] = 30
gen_dict = {1:40, 2:40, 3:50, 4:50, 5:60, 6:60, 7:60, 8:70, 8:70, 9:70, 10: 80, 11:80,
           12:80, 13: 80, 14:90, 15:90}
for key, value in gen_dict.items():
    df_oh.loc[df_oh.PRAEGENDE_JUGENDJAHRE == key, "GENERATION"] = value

In [ ]:
df_oh[["PRAEGENDE_JUGENDJAHRE", "GENERATION"]].head(5)

In [ ]:
df_oh["WEALTH"] = df_oh["CAMEO_INTL_2015"].str[:1]

In [ ]:
df_oh["LIFE_STAGE"] = df_oh["CAMEO_INTL_2015"].str[1:]

In [ ]:
df_oh[["CAMEO_INTL_2015", "WEALTH", "LIFE_STAGE"]].head(5)

In [ ]:
# LP_LEBENSPHASE_GROB is a subset of LP_LEBENSPHASE_FEIN and is therefore dropped
df_oh.drop(columns = "LP_LEBENSPHASE_GROB", inplace = True)

In [ ]:
mixed_oh = ["GENERATION","WEALTH", "LIFE_STAGE",
            "LP_LEBENSPHASE_FEIN", "WOHNLAGE", "KBA05_BAUMAX", "PLZ8_BAUMAX"]

In [ ]:
df_oh = pd.get_dummies(data = df_oh, columns = mixed_oh)

In [ ]:
df_oh.shape

#### Discussion 1.2.2: Engineer Mixed-Type Features
 - The mixed features CAMEO_INTL_2015 and PRAEGENDE_JUGENDJAHRE were transformed as requested
 - LP_LEBENSPHASE_GROB is a subset of LP_LEBENSPHASE_FEIN which is why it is dropped
 - "Wohnlage" is only slightly mixed which is why it is kept
 - KBA05_BAUMAX is almost not mixed which is why it is kept
 - Same for PLZ8_BAUMAX
 - All kept mixed categorical features are also one-hot encoded

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.
drop_cols = ["PRAEGENDE_JUGENDJAHRE", "CAMEO_INTL_2015"]
df_oh.drop(columns = drop_cols, inplace = True)


In [ ]:
df_oh.shape

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
az_df = pd.read_csv("Udacity_AZDIAS_Subset.csv")
cus_df = pd.read_csv("Udacity_CUSTOMERS_Subset.csv")
az_df.columns == cus_df.columns

In [ ]:
def clean_data(df):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    # Load in the feature summary file.
    feat_info = pd.read_csv("AZDIAS_Feature_Summary.csv", delimiter = ";")

    # Identify missing or unknown data values and convert them to NaNs.
    def list_convert(line):
        line = line.replace("[", "")
        line = line.replace("]", "")
        a = line.split(",")
        return a
    feat_info["missing_or_unknown"] = feat_info["missing_or_unknown"].apply(list_convert)
    # Sets the attribute as index so it is easier to handle with the .loc mehtod
    feat_info.set_index("attribute", inplace = True)



    # Iterates through the columns of df
    for column in df.columns:
        # saves the values as a list which are nan in this colum
        null_values = feat_info.loc[str(column), "missing_or_unknown"]
        for entry in null_values:
            # Some columns do not have an indicator for null values,
            try:
                if (entry != ""):
                    # replaces every null value with an np.nan in the column
                    df[column] = df[column].replace(int(entry), np.nan)
                else:
                    print("Entry was an empty string: -{}- (should be empty)".format(entry))
            except:
                print("Entry {} is not a number.".format(entry))
                df[column] = df[column].replace(entry, np.nan)

    nan_columns = ["AGER_TYP", "GEBURTSJAHR", "TITEL_KZ", "ALTER_HH", "HH_EINKOMMEN_SCORE"]
    # drop columns
    df.drop(axis = 1, columns = nan_columns, inplace = True)
    len(df.columns)
    print("Shape of df is: {}".format(df.shape))

    # How much data is missing in each row of the dataset?
    df["missing"] = df.apply(lambda x: (df.shape[1] - x.count()), axis=1)

    df_full = df.query("missing == 0")
    print("Shape is now: {}".format(df_full.shape))

    # How many features are there of each data type?
    cat_mixed_info = feat_info.query("type == ['categorical', 'mixed']")
    cat_columns = list(feat_info.query("type == ['categorical']").index)
    one_hot = []

    for column in cat_columns:
        try:
            print("Column {} has {} unique values".format(column,len(df_full[column].unique())))
            # If it is already one hot encoded do nothing
            if list(df_full[column].unique()) == [0, 1]:
                pass
            # Otherwise append it to a list for one-hot encoding
            else:
                one_hot.append(column)
        except KeyError:
            print("The column {} got removed in an earlier step".format(column))

    df_oh = df_full.copy()
    # Re-encode categorical variable(s) to be kept in the analysis.
    df_oh = pd.get_dummies(data = df_oh, columns = list(one_hot))

    print("Shape of df is now {}".format(df_oh.shape))

    # Engineer new variables
    mainstream = [1,3,5,8,10,12,14]

    df_oh["MAINSTREAM"] = 2
    for number in mainstream:
        df_oh.loc[df_oh.PRAEGENDE_JUGENDJAHRE == number, 'MAINSTREAM'] = "1"
    df_oh["MAINSTREAM"].replace(2, 0, inplace = True)

    df_oh["GENERATION"] = 30
    gen_dict = {1:40, 2:40, 3:50, 4:50, 5:60, 6:60, 7:60, 8:70, 8:70, 9:70, 10: 80, 11:80,
               12:80, 13: 80, 14:90, 15:90}
    for key, value in gen_dict.items():
        df_oh.loc[df_oh.PRAEGENDE_JUGENDJAHRE == key, "GENERATION"] = value

    df_oh["WEALTH"] = df_oh["CAMEO_INTL_2015"].str[:1]
    df_oh["LIFE_STAGE"] = df_oh["CAMEO_INTL_2015"].str[1:]

    # LP_LEBENSPHASE_GROB is a subset of LP_LEBENSPHASE_FEIN and is therefore dropped
    df_oh.drop(columns = ["LP_LEBENSPHASE_GROB"], inplace = True)
    mixed_oh_01 = ["GENERATION","WEALTH", "LIFE_STAGE",
                "LP_LEBENSPHASE_FEIN"]
    mixed_oh_02 = ["WOHNLAGE", "KBA05_BAUMAX", "PLZ8_BAUMAX"]



    df_oh = pd.get_dummies(data = df_oh, columns = mixed_oh_01)
    df_oh["WOHNLAGE_0.0"] = 0
    df_oh = pd.get_dummies(data = df_oh, columns = mixed_oh_02)
    print("The shape is now {}".format(df_oh.shape))

    drop_cols = ["PRAEGENDE_JUGENDJAHRE", "CAMEO_INTL_2015"]
    df_oh.drop(columns = drop_cols, inplace = True)
    
    print("The shape is now {}".format(df_oh.shape))
    
    return df_oh
                                                          

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
feat_info.type.unique()

In [ ]:
feat_info.head()

In [ ]:
# Apply feature scaling to the general population demographics data.
scale_cols = feat_info.query("type == ['ordinal', 'numeric','interval']").index
scale_cols = scale_cols.drop(labels = ['GEBURTSJAHR', 'ALTER_HH', 'HH_EINKOMMEN_SCORE'])

In [ ]:
df_ss = df_oh.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_ss[scale_cols] = scaler.fit_transform(df_ss[scale_cols])

### Discussion 2.1: Apply Feature Scaling

- I already decided to remove rows with missing values since their distribution compared to rows without missing values was significantly differnt
- All non categorical values have been transformed with the standard scaler


### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
df_pca = df_ss.copy()

In [ ]:
# Apply PCA to the data.
from sklearn.decomposition import PCA
pca = PCA()
df_pca = pca.fit_transform(df_pca)

In [ ]:
# Investigate the variance accounted for by each principal component.
cumm_values = []
former = 0
# variable that makes sure the success print is only printed once
first = True
for idx,value in enumerate(pca.explained_variance_ratio_):
    cumm_values.append(value+former)
    former += value
    if former >= 0.9 and first:
        print("{:.2f} % variance explained with {} components".format(former*100, idx))
        first = False
plt.plot(cumm_values);


In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.
pca = PCA(n_components = 70)
df_pca = pca.fit_transform(df_pca)

### Discussion 2.2: Perform Dimensionality Reduction

As seen in the graph above 70 components are enough to explain 90% of the variance. At this point the curve is already very flat. Therefore I will reduce the dimensions of my dataset to 70 components.

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.
comp_df = pd.DataFrame(pca.components_, columns = df_ss.columns)
comp_df.shape

In [ ]:
dimensions = ['Dim_{}'.format(n) for n in range(1, comp_df.shape[0]+1)]
comp_df.index = dimensions

In [ ]:
comp_df.T.Dim_1.sort_values(ascending = False)

In [ ]:
def list_feature_weight(pca, df_ss, n):
    """
    INPUT:
    pca - The trained and transformed pca object
    df_ss - The dataframe on which the pca was applied
    n - the pca dimension to be printed
    
    OUTPUT:
    A series with the sorted feature weights for the requested dimension
    """
    # create a dataframe from the pca component weights for each feature
    comp_df = pd.DataFrame(pca.components_, columns = df_ss.columns)
    # name the index
    dimensions = ['Dim_{}'.format(n) for n in range(1, comp_df.shape[0]+1)]
    comp_df.index = dimensions
    # return the requested dimension
    return comp_df.T["Dim_{}".format(n)].sort_values(ascending = False)
    

In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.
list_feature_weight(pca, df_ss, 2)


In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.

list_feature_weight(pca, df_ss, 3)

### Discussion 2.3: Interpret Principal Components

The first three pca components are all pretty much directly mapped to a single feature. For the first dimension this is the ALTERSKATEGORIE_GROB feature, for the second it is the FINANZ_MINIMALIST, and the third the FINANZ_SPARER. This means that age followed by low financial interest have the highest influence on the dimensions.

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
from sklearn.cluster import KMeans
clusters = [2,4,6,8,10,12,14,16,18,20]

# Over a number of different cluster counts...
kmeans_score = []
for idx,n_cluster in enumerate(clusters):
    print("Fitting kmeans with {} clusters".format(n_cluster))
    # run k-means clustering on the data and...
    kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(df_pca)
    print("Calculating the score...")
    # compute the average within-cluster distances.
    kmeans_score.append(np.abs(kmeans.score(df_pca)))
    
    

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.
plt.plot(clusters,kmeans_score)


In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.
kmeans = KMeans(n_clusters=18, random_state=0).fit(df_pca)


In [ ]:
preds = kmeans.predict(df_pca)
df_ss["cluster"] = preds
df_oh["cluster"] = preds

### Discussion 3.1: Apply Clustering to General Population

As the scree plot above shows there isn't a clear elbow possibly due to the still large amount of dimensions of the pca reduction. However after 18 clusters the scree plot is very flat, which is why I decided to go with 18 clusters.

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customers = pd.read_csv("Udacity_CUSTOMERS_Subset.csv", delimiter = ";")

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.
cus_df = clean_data(customers)


In [ ]:
scale_cols = feat_info.query("type == ['ordinal', 'numeric','interval']").index
scale_cols = scale_cols.drop(labels = ['GEBURTSJAHR', 'ALTER_HH', 'HH_EINKOMMEN_SCORE'])
cus_ss = cus_df.copy()
cus_ss[scale_cols] = scaler.transform(cus_ss[scale_cols])

In [ ]:
cus_pca = pca.transform(cus_ss)

In [ ]:
cus_df["cluster"] = kmeans.predict(cus_pca)

In [ ]:
cus_df.head(5)

### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.
cus = cus_df.copy()
pop = df_oh.copy()

In [ ]:
plt.countplot(x = "cluster",data = pop, color = "grey" )

In [ ]:
sns.countplot(x = "cluster",data = cus, color = "grey" )

In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?
# Customers are mostly in cluster 1

# Invert the pca transformation of the cluster centers to be able to easier map them to single columns
cluster_centers = kmeans.cluster_centers_
cc_ss = pca.inverse_transform(cluster_centers)

In [ ]:
# Invert the scaling to the original scaling for easier interpretation
cc_org = pd.DataFrame(cc_ss, columns = cus_ss.columns)
#cc_org[scale_cols] = scaler.transform(cus_org[scale_cols])

In [ ]:
# The cluster most customers belong to is "1"
# Listed below are the dimensions with the most positive correlation
cc_org.iloc[1:2,:].T.sort_values(by = 1, ascending = False)[:4]

In [ ]:
# Listed below are the dimensions with the negative correlation
cc_org.iloc[1:2,:].T.sort_values(by = 1, ascending = True)[:4]

In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?
cc_org.iloc[15:16,:].T.sort_values(by = 15, ascending = False)[:4]


In [ ]:
cc_org.iloc[15:16,:].T.sort_values(by = 15, ascending = True)[:4]

### Discussion 3.3: Compare Customer Data to Demographics Data
##### Biggest cluster of customers
The cluster "1" has by far the most customers in it. It is largely over representated in comparison with the cluster in the general population.
The biggest factor of the cluster is the dimension "FINANZ_MINIMALIST" with a positive correlation. In addition with a negative correlation of "FINANZ_SPARER" and "FINANZ_UNAUFFAELLIGER" we get the picture that the biggest customer cluster is someone with the financial type of a saver of inconspicuous but who is not a financal minimalist (since a large number in these columns correlates with a low association).

Another characteristic of the customer is his health type which is "critical reserved".

##### Smallest cluster of customers
In cluster 15 we have almost no customers of the company but a large share of the general population.
If we look into the position of the centroid of this cluster we see that this consists of people who strongly identify as FINANZ_MINIMALIST (low score means "very high") and also say they do not belong to the type "SPARER" of financial type. This makes perfect sense since it is pretty much the opposite of the cluster with the most customers.


> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.